# IMPORTANDO BIBLIOTECAS

In [2]:
import pandas as pd
import joblib
import time
import os
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

print("Bibliotecas carregadas!")

Bibliotecas carregadas!


### Download das stopwords em português (“de”, “a”, “o”, “que”, “e”, “do”, “da”, “em”, “um”, “para” etc.)

In [3]:
try:
    stopwords_pt = stopwords.words('portuguese')
except:
    print("Baixando stopwords do NLTK...")
    nltk.download('stopwords')
    stopwords_pt = stopwords.words('portuguese')

### Variáveis globais, nome do arquivo e conexão com o banco de dados

In [4]:
BASE_DIR = Path.cwd()
env_path = BASE_DIR.parent / '.env'
load_dotenv(dotenv_path=env_path)

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = "localhost"
port = "5432"
db_name = os.getenv("DB_NAME")

DB_URL = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

modelo_achv = "modelo_recomendacao.pkl"

try:
    engine = create_engine(DB_URL)
    print("Conexão com o banco de dados criada com sucesso!")
except Exception as e:
    print(f"Erro ao criar conexão com o banco: {e}")
print(f"Modelo será salvo em: {modelo_achv}")

Conexão com o banco de dados criada com sucesso!
Modelo será salvo em: modelo_recomendacao.pkl


### Carregar os filmes do bd e criação do dataframe para manipulação dos textos

#### Removi filmes onde a quantidade de votos era menor que 50 (qtd_votos > 50) pois filmes com pouca relevância e avaliações estavam criando "ruídos" e distorcendo os dados para os filmes na recomendação, recomendando filmes com pouca correlação contextual

In [5]:
try:
    print("CARREGANDO FILMES DO BD")
    start_time = time.time()
    
    query = """
            SELECT
                tmdb_id, titulo, sinopse, generos,
                elenco, diretor, keywords, media_votos
            FROM filmes
            WHERE qtd_votos > 50
            """
    df_filmes = pd.read_sql(query, engine)
    
    print(f"{len(df_filmes)} carregados em {time.time() - start_time:.2f}s")
    print("Processamento de textos a seguir...")

except Exception as e:
    print("VERIFICAR A CONEXÃO COM O BANCO DE DADOS E BIBLIOTECAS INSTALADAS")
    print(e)

CARREGANDO FILMES DO BD
4665 carregados em 0.08s
Processamento de textos a seguir...


### Visualizar os dados brutos retirados do bd no nosso dataframe

In [7]:
df_filmes.head()

,tmdb_id,titulo,sinopse,generos,elenco,diretor,keywords,media_votos
0,19165,Em Busca do Vale Encantado V: A Ilha Misteriosa,Além do Vale Encantado existe uma ilha de bele...,"Animação,Família,Aventura","John Ingle,Brandon La Croix,Aria Noelle Curzon...",Charles Grosvenor,dinosaur,6.200
1,8069,Barbarella,"No século XLI, as guerras já foram abolidas há...","Ficção científica,Aventura,Comédia,Fantasia,Ação","Jane Fonda,John Phillip Law,Anita Pallenberg,M...",Roger Vadim,"angel,sexual fantasy,alien planet,distant futu...",5.982
2,288708,Im Keller,,Documentário,"Alessa Duchek,Gerald Duchek,Inge Ellinger,Manf...",Ulrich Seidl,"cellar,austria,basement",6.157
3,388,O Plano Perfeito,Assaltantes vestidos com uniformes de pintor i...,"Crime,Drama,Thriller","Denzel Washington,Clive Owen,Jodie Foster,Chri...",Spike Lee,"new york city,nazi,court case,kidnapping,hosta...",7.399
4,1257960,Sikandar,Um trágico acidente leva o poderoso Sikandar a...,"Ação,Drama","Salman Khan,Rashmika Mandanna,Sathyaraj,Sharma...",A.R. Murugadoss,,4.900


### Definição de funções para auxiliar com textos, ex: "nome sobrenome" vira "nomesobrenome" e "o filme de tal" vira "filme tal" (modelo usa a unica entidade "nomesobrenome", melhor que duas entidades "nome" e "sobrenome"

In [8]:
def limpar_nomes(nomes_str):
    """ Pega os 3 primeiros nomes, remove espaços e junta. """
    if not isinstance(nomes_str, str):
        return ""
    nomes_limpos = [n.strip().lower().replace(" ", "") for n in nomes_str.split(",")[:3]]
    return ' '.join(nomes_limpos)


def limpar_texto(texto):
    """ Remove stopwords do texto. """
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    # Uma forma um pouco mais robusta de remover stopwords
    palavras = texto.split()
    palavras_sem_stopwords = [p for p in palavras if p not in stopwords_pt]
    return ' '.join(palavras_sem_stopwords)

print("Funções de limpeza definidas.")

Funções de limpeza definidas.


### Realização da limpeza dos textos e remoção das stopwords utilizando as funções auxiliares anteriores

#### Não realizei a limpeza no "título" do filme para evitar recomendações de sequências de forma incessante, em testes anteriores "O Poderoso Chefão" recomendava diretamente "O Poderoso Chefão 2" com um grande score apenas pelo nome igual.

In [9]:
print("Iniciando processamento e limpeza dos dados...")

# Copiar para evitar SettingWithCopyWarning
df = df_filmes.copy() 

df = df.fillna("")
df["elenco_limpo"] = df["elenco"].apply(limpar_nomes)
df["diretor_limpo"] = df["diretor"].apply(limpar_nomes)
df["keywords_limpas"] = df["keywords"].str.replace(",", " ", regex=False)
df["generos"] = df["generos"].str.replace(",", " ", regex=False)

print("Processamento inicial concluído.")

Iniciando processamento e limpeza dos dados...
Processamento inicial concluído.


### Visualizar dataframe com formatação

In [10]:
df[['titulo', 'elenco_limpo', 'diretor_limpo', 'keywords_limpas']].head()

,titulo,elenco_limpo,diretor_limpo,keywords_limpas
0,Em Busca do Vale Encantado V: A Ilha Misteriosa,johningle brandonlacroix arianoellecurzon,charlesgrosvenor,dinosaur
1,Barbarella,janefonda johnphilliplaw anitapallenberg,rogervadim,angel sexual fantasy alien planet distant futu...
2,Im Keller,alessaduchek geraldduchek ingeellinger,ulrichseidl,cellar austria basement
3,O Plano Perfeito,denzelwashington cliveowen jodiefoster,spikelee,new york city nazi court case kidnapping hosta...
4,Sikandar,salmankhan rashmikamandanna sathyaraj,a.r.murugadoss,


### Criação da ordem semântica que será utilizado pelo modelo

#### Respeito do limite de tokens do modelo, após pesquisas e novas análises, deixando de ser um "buscador de palavras chaves" e recomendando de fato buscando contexto e semântica, para melhor recomendação híbrida.

In [12]:
print("Construindo texto combinado com pesos...")

df["texto"] = (
    "Temas: " + df["keywords_limpas"].fillna("") + "\n" +
    "Gêneros: " + df["generos"].fillna("") + "\n" +
    "Direção: " + df["diretor_limpo"].fillna("") + "\n" +
    "Sinope: " + df["sinopse"].fillna("") + "\n" +
    "Elenco: " + df["elenco_limpo"].fillna("") + "\n" +
    "Título: " + df["titulo"].fillna("")
)

print("Aplicando limpeza de stopwords no texto combinado...")
df['texto'] = df['texto'].apply(limpar_texto)

print("Campo 'texto' finalizado.")

Construindo texto combinado com pesos...
Aplicando limpeza de stopwords no texto combinado...
Campo 'texto' finalizado.


### Visualização de resultado de um índice como exemplo;

In [13]:
df['texto'].iloc[0]

'temas: dinosaur gêneros: animação família aventura direção: charlesgrosvenor sinope: além vale encantado existe ilha beleza mistério. neste lugar maravilhoso, littlefoot, saura, espora, patassaura petrúcio descobrirão velhos novos amigos, enfrentarão incríveis desafios viverão maior aventura vidas! nova história recheada canções clássico busca vale encantado. dia, nuvem insetos devoradores plantas desce sobre vale encantado, acabam todas folhinhas vista. agora, vale totalmente destruído todos precisam buscar novo lar plantas cresçam novamente. brigas adultos ameaçam separa-los, littlefoot amigos resolvem partir sozinhos. busca, encontram, grandes águas – onde estranhas criaturas vivem nadam – levam misteriosa ilha onde encontram chomper, adorável amigo dentes afiados “a grande aventura vale”. porém, agora chomper precisa proteger amiguinhos feras, incluindo próprios país!. elenco: johningle brandonlacroix arianoellecurzon título: busca vale encantado v: ilha misteriosa'

#### No dataframe acima é possível verificar o texto criado com os parâmetros anteriores

### Carregando o modelo de lingaguem utilizado

In [14]:
print("Carregando modelo de linguagem (SentenceTransformer)...")
start_model_load = time.time()

modelo = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

print(f"Modelo carregado em {time.time() - start_model_load:.2f}s")

Carregando modelo de linguagem (SentenceTransformer)...
Modelo carregado em 2.96s


### Geração de embeddings do modelo, o "treinamento", transformação das nossas colunas em vetores numéricos e utilizando todos os vetores com comprimento 1 para um cálculo mais rápido e correto na similaridade de cossenos

In [15]:
print("Gerando embeddings semânticos...")
start_embed = time.time()

embeddings = modelo.encode(
    df['texto'].tolist(),
    show_progress_bar=True,
    normalize_embeddings=True
)

print(f"Embeddings gerados em {time.time() - start_embed:.2f}s")
print(f"Formato dos embeddings (Shape): {embeddings.shape}")

Gerando embeddings semânticos...


Batches: 100%|███████████████████████████████████████████████████████████████████████| 146/146 [01:38<00:00,  1.48it/s]

Embeddings gerados em 98.90s
Formato dos embeddings (Shape): (4665, 384)


### Salvando os artefatos do modelo treinado em pkl, metadata de inforamções úteis para utilização no script de recomendação

In [16]:
print('Salvando artefatos do modelo de ML...')

artefatos = {
    "version": "1.3.1",
    "date": time.strftime("%Y-%m-%d %H:%M:%S"),
    "model_name": "paraphrase-multilingual-MiniLM-L12-v2",
    "embeddings": embeddings,
    "tmdb_ids": df["tmdb_id"].tolist(),
    "metadata": df[["tmdb_id", "titulo", "media_votos"]].to_dict(orient="records")
}

joblib.dump(artefatos, modelo_achv)
print(f"Modelo salvo em '{modelo_achv}' com {len(df)} filmes")
print("TREINAMENTO CONCLUÍDO!")

Salvando artefatos do modelo de ML...
Modelo salvo em 'modelo_recomendacao.pkl' com 4665 filmes
TREINAMENTO CONCLUÍDO!
